In [1]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gc
import os
from tqdm import tqdm
import numpy as np

warnings.simplefilter('ignore')
%matplotlib inline

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
seed = 2020

In [3]:
# !pip install xgboost

In [4]:
df_feature = pd.read_pickle('data/feature.pkl')

In [5]:
df_feature.head()

id  loanAmnt  term  interestRate  installment  grade  subGrade  \
0   1   18000.0     5         18.49       461.90      4        21   
1  23   20000.0     3         14.08       684.33      3        17   
2  28    8425.0     3          8.59       266.31      1         9   
3  32   10000.0     3         15.59       349.55      4        20   
4  42   13000.0     5         13.35       298.13      3        16   

   employmentTitle  employmentLength  homeOwnership  annualIncome  \
0         219843.0               5.0              0       46000.0   
1            386.0               4.0              1       47500.0   
2           2780.0              10.0              0       40000.0   
3            132.0               5.0              2       30000.0   
4          81832.0              10.0              1       52000.0   

   verificationStatus   issueDate  isDefault  purpose  postCode  regionCode  \
0                   2  2012-08-01        0.0        0     156.0          18   
1                   1  2018-01-01        1.0        9     226.0          21   
2                   0  2016-08-01        0.0        0     245.0          26   
3                   1  2015-01-01        0.0        0     413.0          37   
4                   1  2014-10-01        1.0        4     524.0          20   

     dti  delinquency_2years  ficoRangeLow  ficoRangeHigh  openAcc  pubRec  \
0  27.83                 0.0         700.0          704.0     13.0     0.0   
1  24.36                 0.0         665.0          669.0      7.0     0.0   
2  12.57                 2.0         665.0          669.0     17.0     1.0   
3  20.20                 0.0         685.0          689.0      7.0     0.0   
4  18.44                 0.0         705.0          709.0      7.0     0.0   

   pubRecBankruptcies  revolBal  revolUtil  totalAcc  initialListStatus  \
0                 0.0   15096.0       38.9      18.0                  1   
1                 0.0   19497.0       53.1      19.0                  0   
2                 1.0    7993.0       33.9      23.0                  0   
3                 0.0    6772.0       41.3      17.0                  1   
4                 0.0   17703.0       38.8      13.0                  1   

   applicationType earliesCreditLine   title   n0   n1   n2   n3    n4   n5  \
0                0          May-2002  1723.0  NaN  NaN  NaN  NaN  10.0  NaN   
1                0          Mar-2000    10.0  0.0  4.0  4.0  4.0   4.0  7.0   
2                0          Apr-2000     0.0  0.0  5.0  9.0  9.0   8.0  8.0   
3                0          Jul-2007     0.0  0.0  2.0  3.0  3.0   3.0  4.0   
4                0          Jul-2004     4.0  0.0  3.0  5.0  5.0   4.0  5.0   

    n6    n7    n8   n9   n10  n11  n12  n13  n14  issueDate_year  \
0  NaN   NaN   NaN  NaN  13.0  NaN  NaN  NaN  NaN            2012   
1  9.0   6.0  10.0  4.0   7.0  0.0  0.0  0.0  1.0            2018   
2  5.0  17.0  18.0  9.0  17.0  0.0  0.0  0.0  1.0            2016   
3  9.0   6.0   8.0  3.0   7.0  0.0  0.0  0.0  4.0            2015   
4  5.0   6.0   8.0  5.0   7.0  0.0  0.0  0.0  1.0            2014   

   earliesCreditLine_year  issueDate_year_earliesCreditLine_year_minus  \
0                    2002                                           10   
1                    2000                                           18   
2                    2000                                           16   
3                    2007                                            8   
4                    2004                                           10   

   debt_ratio_year  applicationType_cnt  employmentLength_cnt  \
0         0.078261               980693               62645.0   
1         0.140351               980693               59818.0   
2         0.070208               980693              328525.0   
3         0.111111               980693               62645.0   
4         0.050000               980693              328525.0   

   employmentTitle_cnt  grade_cnt  homeOwnership_cnt

In [6]:
for f in df_feature.select_dtypes('object').columns:
    lbl = LabelEncoder()
    df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

In [7]:
df_train = df_feature[df_feature.isDefault.notna()].copy()
df_test = df_feature[df_feature.isDefault.isna()].copy()

df_train.shape, df_test.shape

((800000, 338), (200000, 338))

In [8]:
ycol = 'isDefault'
feature_names = list(
    filter(lambda x: x not in [ycol, 'id'], df_train.columns))

model = xgb.XGBClassifier(max_depth=6,
                          learning_rate=0.05,
                          n_estimators=10000,
                          subsample=0.8,
                          reg_alpha=10,
                          reg_lambda=12,
#                           tree_method='gpu_hist',
                          random_state=seed)

df_oof = df_train[['id', ycol]].copy()
df_oof['prob'] = 0
prediction = df_test[['id']]
prediction['prob'] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))

    xgb_model = model.fit(X_train,
                          Y_train,
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=100,
                          eval_metric='auc',
                          early_stopping_rounds=50)

    pred_val = xgb_model.predict_proba(X_val)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val

    pred_test = xgb_model.predict_proba(df_test[feature_names])[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': xgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del xgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

[0]	validation_0-auc:0.70767	validation_1-auc:0.70730
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[100]	validation_0-auc:0.74134	validation_1-auc:0.73572
[200]	validation_0-auc:0.75118	validation_1-auc:0.74072
[300]	validation_0-auc:0.75815	validation_1-auc:0.74286
[400]	validation_0-auc:0.76410	validation_1-auc:0.74402
[500]	validation_0-auc:0.76940	validation_1-auc:0.74448
[600]	validation_0-auc:0.77444	validation_1-auc:0.74488
[700]	validation_0-auc:0.77939	validation_1-auc:0.74508
[800]	validation_0-auc:0.78400	validation_1-auc:0.74523
[900]	validation_0-auc:0.78857	validation_1-auc:0.74535
Stopping. Best iteration:
[920]	validation_0-auc:0.78946	validation_1-auc:0.74538


Fold_2 Training ================================

[0]	validation_0-auc:0.70760	validation_1-auc:0.70470
Multiple eval metrics have been passed: 'valid

In [9]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby([
    'column'
])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

column  importance
0                 homeOwnership_subGrade_default_ratio    0.289090
1                    regionCode_subGrade_default_ratio    0.044752
2                       grade_regionCode_default_ratio    0.039959
3                                                 term    0.027606
4              employmentLength_subGrade_default_ratio    0.026400
5                    grade_homeOwnership_default_ratio    0.014861
6                         subGrade_title_default_ratio    0.013016
7             initialListStatus_subGrade_default_ratio    0.011787
8                                            issueDate    0.011570
9                                                  n11    0.011009
10                           grade_title_default_ratio    0.010448
11                                                  n2    0.008679
12                       employmentTitle_default_ratio    0.008644
13                                        ficoRangeLow    0.007727
14              employmentTitle_postCode_default_ratio    0.007190
15                                 employmentTitle_cnt    0.006808
16                                                 n14    0.006581
17                                     debt_ratio_year    0.006059
18                homeOwnership_postCode_default_ratio    0.006045
19       applicationType_employmentTitle_default_ratio    0.005479
20                                                  n9    0.005414
21                                                 dti    0.005382
22              homeOwnership_regionCode_default_ratio    0.005373
23                      employmentTitle_regionCode_cnt    0.005223
24                employmentLength_grade_default_ratio    0.004974
25         employmentTitle_homeOwnership_default_ratio    0.004930
26                                            loanAmnt    0.004505
27                   applicationType_homeOwnership_cnt    0.004283
28                                         purpose_cnt    0.004154
29                                  delinquency_2years    0.004071
30                                            revolBal    0.003994
31              employmentTitle_verificationStatus_cnt    0.003991
32            employmentTitle_regionCode_default_ratio    0.003881
33                   homeOwnership_title_default_ratio    0.003849
34                         applicationType_purpose_cnt    0.003834
35                              verificationStatus_cnt    0.003704
36           subGrade_verificationStatus_default_ratio    0.003582
37                                         installment    0.003538
38               employmentTitle_initialListStatus_cnt    0.003528
39              title_verificationStatus_default_ratio    0.003508
40                                        annualIncome    0.003468
41                        employmentTitle_postCode_cnt    0.003388
42                 applicationType_employmentTitle_cnt    0.003294
43                                        interestRate    0.003166
44                   employmentTitle_homeOwnership_cnt    0.003164
45                                  pubRecBankruptcies    0.003159
46         issueDate_year_earliesCreditLine_year_minus    0.003138
47                                                  n8    0.003127
48               applicationType_purpose_default_ratio    0.003044
49                       initialListStatus_purpose_cnt    0.002940
50                          employmentLength_title_cnt    0.002921
51                                              pubRec    0.002891
52                        applicationType_subGrade_cnt    0.002886
53                           applicationType_title_cnt    0.002852
54                employmentLength_title_default_ratio    0.002835
55                                    employmentLength    0.002833
56                      purpose_subGrade_default_ratio    0.002829
57              grade_verificationStatus_default_ratio    0.002826
58     employmentTitle_initialListStatus_default_ratio    0.002826
59    employmentTitle_verifi

In [10]:
auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
print('auc:', auc)

auc: 0.743330876642909


In [11]:
os.makedirs('sub', exist_ok=True)

sub = prediction[['id', 'prob']].copy()
sub.columns = ['id', 'isDefault']
sub.to_csv('sub/jiedai_{}.csv'.format(auc), index=False)

In [12]:
os.makedirs('prob', exist_ok=True)

prediction.to_csv('prob/sub_xgb.csv', index=False)
df_oof[['id', 'prob', ycol]].to_csv('prob/oof_xgb.csv', index=False)

prediction.to_csv('prob/sub_xgb_{}.csv'.format(auc), index=False)
df_oof[['id', 'prob', ycol]].to_csv(
    'prob/oof_xgb_{}.csv'.format(auc), index=False)

In [13]:
df_oof.head()

id  isDefault      prob
0   1        0.0  0.298526
1  23        1.0  0.341080
2  28        0.0  0.115245
3  32        0.0  0.278753
4  42        1.0  0.295865

In [14]:
prediction.head()

id      prob
800000  800000  0.052258
800001  800001  0.308782
800002  800002  0.264748
800003  800003  0.299554
800004  800004  0.363835